In [1]:
# importing all the modules
import glob
import os
import tabula
import pandas as pd
import numpy as np
from os import listdir


In [2]:
# to create a list contaning all the df obtained from pdf structure: [[df_1,_df_2], [df_1,df_2]]
#names = [str(i) for i in range(1,3)]
#pdfs = []
#for pdf in names:
 #   file_name = "%s.pdf" % pdf
  #  df =  tabula.read_pdf(file_name, pages = 2)
#pdfs.append(df)

In [4]:
my_dir= "C:\\Users\\bhaveshthakkar\\Study\\Files\\"
pdf_files = glob.glob(os.path.join(my_dir, '*.pdf'))

all_pdfs = []
for pdfs in pdf_files:
    file_name = pdfs
    df =  tabula.read_pdf(file_name, pages = 2)
    all_pdfs.append(df)


In [5]:
#selecting lists which has only three data frames  (length of orignal list is more than 100 of which majority sublists
# contains 3 data frames)
new_list = []
for i in all_pdfs:
    if len(i) == 3:
        new_list.append(i)
        
        

In [6]:
#To concat all the data frames in each sublists
all_df_1 = []
for i in new_list:
    all_df_1.append(pd.concat([i[0],i[1],i[2]], axis = 0, ignore_index = True ))

In [8]:
# Operating on all the data frames in the previously concated list of dfs  
all_df_final = []

for i in range(len(all_df_1)):
    df_1 = all_df_1[i][all_df_t[i]["Unnamed: 0"].notna()]
    df_1 = df_1.iloc[~df_1.index.isin([1,2])]
    df_2 = df_1.iloc[:,0:3]
    df_2 = df_2.T
    df_2.columns = df_2.iloc[0]
    df_2 = df_2.iloc[1:]
    df_2 = df_2.fillna(method = "bfill")
    df_2 = df_2.iloc[[0]]
    df_2 = df_2.iloc[:,0:5]
    df_3 = df_1.iloc[:,4:].dropna(how = "all")
    df_3.columns = df_3.iloc[0]
    df_3 = df_3.iloc[1:]
    df_1 = pd.concat([df_2, df_3], axis = 1)
    df_1 = df_1.fillna(method = "ffill").iloc[1:]
    all_df_final.append(df_1)
    


In [199]:
#creating a list from the all_pdfs for sublists with 3 dfs each
new_list_2 = []
for i in all_pdfs:
    if len(i) != 3:
        new_list_2.append(i)

In [161]:
new_list_2[1]

[     Unnamed: 0                         Graduate Group Information  Unnamed: 1
 0  Organization  School of Criminology and Criminal Justice Gra...         NaN
 1  Contact Name                                     Sarah Lockwood         NaN
 2         Email                           lockwood.s@husky.neu.edu         NaN
 3  Award Amount                                               $210         NaN,
           Unnamed: 0                                  Event Information  \
 0         Event Name  Research and Identity Lunch Roundtable Discussion   
 1         Event Date                                         11/21/2019   
 2  Event Description  Thepurposeofthiseventistogatherstudentsandfacu...   
 
    Unnamed: 1  
 0         NaN  
 1         NaN  
 2         NaN  ,
    Transaction Detail Unnamed: 0 Unnamed: 1 Unnamed: 2   Unnamed: 3 Unnamed: 4
 0  Transaction Detail        NaN        NaN        NaN          NaN        NaN
 1                From         To        NaN        NaN         

In [201]:
# concating all dfs in each sublists
all_df_2 = []
for i in new_list_2:
    all_df_2.append(pd.concat(i, axis = 0, ignore_index = True ).iloc[:,0:9])

In [410]:
# Operating on dfs inside he sublist
final_list = []

for i in range(len(all_df_2)):
    df = all_df_2[i].drop(columns = ["From"]).dropna(how = "all").reset_index(drop = True)
    df = df[~df["Transaction Detail"].isin(["Transaction Detail", "From"])]
    df = df.T
    df.columns = df.iloc[0]
    df = df.reset_index(drop = True).drop(columns = ["Account", "Account"]).iloc[1:,:]#.fillna(method = "bfill")
    df_1 = df.iloc[0:3,:].dropna(axis = 1, how = "all").fillna(method = "bfill").iloc[[0],:].rename(columns = {"73000":"index"})
    df_2 = df.iloc[:,7:].dropna(how = "all")
    df_2 = df_2.T.drop(columns = 4).reset_index(drop = True)
    df = pd.concat([df_1,df_2], axis = 1).drop(columns = ["Email", "Contact Name"])
    final_list.append(df)

In [350]:
# Generating CSV for all the data frame in list
# Looping over all dfs to create a concate df. 
# Did not concat all dfs before exporting into CSV as there were few data frames which had diffrent number on columns

writer = pd.ExcelWriter('final_1.xlsx', engine='xlsxwriter')

# Add the first dataframe to the worksheet.
all_df_final[0].to_excel(writer, sheet_name='Sheet1', index=False)

offset = len(all_df_final[0]) + 1  # Add extra row for column header.

# Add the other dataframes.
for df in all_df_final:
    # Write the datafram without a column header or index.
    df.to_excel(writer, sheet_name='Sheet1', startrow=offset,
                header=False, index=False)

    offset += len(df)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [422]:
writer = pd.ExcelWriter('final_2.xlsx', engine='xlsxwriter')

# Add the first dataframe to the worksheet.
final_list[0].to_excel(writer, sheet_name='Sheet1', index=True)

offset = len(final_list[0]) + 1  # Add extra row for column header.

# Add the other dataframes.
for df in all_df_final:
    # Write the datafram without a column header or index.
    df.to_excel(writer, sheet_name='Sheet1', startrow=offset,
                header=False, index=False)

    offset += len(df)

# Close the Pandas Excel writer and output the Excel file.
writer.save()